<a href="https://colab.research.google.com/github/jhonix007/test-demo/blob/master/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Установка pyspark с помощью pip

In [1]:
pip install --quiet pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Practice").getOrCreate()

## Скачиваем файл

In [3]:
from pyspark import SparkFiles

covid_data_file_url = "https://raw.githubusercontent.com/glincow/netology-spark-sql/main/data/covid-data.csv"
spark.sparkContext.addFile(covid_data_file_url)
file_path  = 'file://' + SparkFiles.get('covid-data.csv')
df = spark.read.option('inferSchema', 'true').option('header', 'true').csv(file_path)

Выберите 15 стран с наибольшим процентом переболевших на 31 марта (в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших)

In [9]:
from pyspark.sql import functions as F
from pyspark.sql.functions import row_number, desc
from pyspark.sql.functions import row_number,lit
from pyspark.sql.window import Window

In [7]:
df_cases_in_march = df.select("iso_code", "location", (F.col('total_cases_per_million') / 1000).alias("total_per_cent_of_sick")).sort(F.col("total_per_cent_of_sick").desc()).where(F.col('date')=='2021-03-31')
df_cases_in_march.show(15)

+--------+-------------+----------------------+
|iso_code|     location|total_per_cent_of_sick|
+--------+-------------+----------------------+
|     AND|      Andorra|            155.439073|
|     MNE|   Montenegro|    145.23725399999998|
|     CZE|      Czechia|            143.088484|
|     SMR|   San Marino|            139.371796|
|     SVN|     Slovenia|    103.70805800000001|
|     LUX|   Luxembourg|             98.473424|
|     ISR|       Israel|              96.25106|
|     USA|United States|     92.03011000000001|
|     SRB|       Serbia|             88.263286|
|     BHR|      Bahrain|             84.888601|
|     PAN|       Panama|             82.287391|
|     PRT|     Portugal|             80.586997|
|     EST|      Estonia|             80.226816|
|     SWE|       Sweden|     79.69744299999999|
|     LTU|    Lithuania|     79.38864699999999|
+--------+-------------+----------------------+
only showing top 15 rows



Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном порядке по убыванию
(в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)

In [10]:
dates = ('2021-03-25', '2021-03-31')
df_max_new_cases = df.filter(F.col('date').between(*dates)).select("date",
                                                                    "location"
                                                                    ,"new_cases"


).sort(F.col("new_cases").desc()).where((F.col('location')!='World') &
                                        (F.col('location')!='Europe') &
                                        (F.col('location')!='Asia') &
                                        (F.col('location')!= 'North America') &
                                        (F.col('location')!='European Union') &
                                        (F.col('location')!='South America') &
                                        (F.col('location')!='Africa'))

# Применим partitionBy() and orderBy()
window_spec = Window.partitionBy("location").orderBy(desc("new_cases"))

# Добавит столбец "row_number" используя row_number()
result_df = df_max_new_cases.withColumn("row_number", row_number().over(window_spec)).where((F.col("row_number")==1))

result_df.select("date", "location", "new_cases").sort(F.col("new_cases").desc()).show(10)

+----------+-------------+---------+
|      date|     location|new_cases|
+----------+-------------+---------+
|2021-03-25|       Brazil| 100158.0|
|2021-03-26|United States|  77321.0|
|2021-03-31|        India|  72330.0|
|2021-03-31|       France|  59054.0|
|2021-03-31|       Turkey|  39302.0|
|2021-03-26|       Poland|  35145.0|
|2021-03-31|      Germany|  25014.0|
|2021-03-26|        Italy|  24076.0|
|2021-03-25|         Peru|  19206.0|
|2021-03-26|      Ukraine|  18226.0|
+----------+-------------+---------+
only showing top 10 rows



Посчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. (например: в россии вчера было 9150 , сегодня 8763, итог: -387) (в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, кол-во новых случаев сегодня, дельта)

In [18]:
w = Window.orderBy("Date")
dates = ('2021-03-24', '2021-03-31')

w = Window().orderBy("date")



df_changes = df.filter(F.col('date').between(*dates)).where(F.col('location').startswith("Russia")).select(
                                                                                                  "date","location", (F.col('new_cases')).alias("new_cases_today"),
                                                                                                   F.lag("new_cases").over(w).alias("new_cases_yestarday")
                                                                                                         ).where(F.col('location').startswith("Russia"))


df_changes = df_changes.select("date",
                  "new_cases_today",
                  "new_cases_yestarday",
                  (F.col('new_cases_today')-(F.col('new_cases_yestarday')).alias("delta"))
                 )


df_changes.withColumnRenamed("(new_cases_today - new_cases_yestarday AS delta)", "delta").filter(F.col('new_cases_yestarday').isNotNull()).show()

+----------+---------------+-------------------+------+
|      date|new_cases_today|new_cases_yestarday| delta|
+----------+---------------+-------------------+------+
|2021-03-25|         9128.0|             8769.0| 359.0|
|2021-03-26|         9073.0|             9128.0| -55.0|
|2021-03-27|         8783.0|             9073.0|-290.0|
|2021-03-28|         8979.0|             8783.0| 196.0|
|2021-03-29|         8589.0|             8979.0|-390.0|
|2021-03-30|         8162.0|             8589.0|-427.0|
|2021-03-31|         8156.0|             8162.0|  -6.0|
+----------+---------------+-------------------+------+

